<a href="https://colab.research.google.com/github/Surafeljava/asa-training/blob/main/AmBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_p0uwtoa
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-_p0uwtoa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 14.2 MB/s 
     |████████████████████████████████| 596 kB 88.5 MB/s 
     |████████████████████████████████| 6.6 MB 80.7 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4004577 sha256=e789be4b4c8c3837ec67d95a992d53053b7ade40ac274ae4667276ffe88d4390
  Stored in directory: /tmp/pip-ephem-wheel-cache-ybu6pfvw/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uni

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1. Tokenizing out Amharic dataset using CharBPETokenizer**

In [ ]:
%%time 
# from pathlib import Path
#Training a custom tokenizer with CharBPETokenizer

from tokenizers import CharBPETokenizer

# Initialize a tokenizer
tokenizer = CharBPETokenizer()

# danny = './drive/MyDrive/amh_data_2.txt'
# eyob = "./drive/MyDrive/bigdata/amh_small/amh_data_2.txt"
# samia = "./drive/MyDrive/amh_data.txt"
# ahlam = "./drive/MyDrive/amh_data.txt"
surafel_link = "./drive/MyDrive/Amharic_Sentiment_Analysis/Dataset/amh_data.txt"

# Then train it!
tokenizer.train([surafel_link], vocab_size=52_000, min_frequency=1, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Now, let's use it:
# encoded = tokenizer.encode("የአየር አካል ግድየለሽ ፣ ተስማሚ እና በስሜታዊነት የተገናኘ ግለሰብን ያመለክታል።")

# print(encoded.tokens)

CPU times: user 2min 45s, sys: 17.7 s, total: 3min 3s
Wall time: 32.9 s


In [ ]:
# And finally save it somewhere
tokenizer.save("./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT/tokenizer.json")
tokenizer.save_model('./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT')

['./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT/vocab.json',
 './drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT/merges.txt']

In [ ]:
from tokenizers import CharBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = CharBPETokenizer(
    "./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT/vocab.json",
    "./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("ማረሚያ ቤቶች አስተዳደር")

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("ማረሚያ ቤቶች አስተዳደር").tokens

['<s>', 'ማረሚያ</w>', 'ቤቶች</w>', 'አስተዳደር</w>', '</s>']

**2. Training Our Language Model using RoBERTa Architecture**

In [ ]:
!nvidia-smi

Sun Apr 17 10:10:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT", max_len=512)

In [ ]:
tokenizer.encode("ማረሚያ ቤቶች አስተዳደር")

[14647, 2882, 3028]

In [ ]:
#Initializing our model
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

83504416

In [ ]:
#Building our training dataset
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=surafel_link,
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 4min 26s, sys: 7.32 s, total: 4min 33s
Wall time: 1min 2s


In [ ]:
#Creating a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform 'BACKPROP' on.
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

**3. Initializing our trainer**

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

**Starting our training :)**

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 620743
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 9700
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is p

Step,Training Loss
500,8.975300
1000,8.346300
1500,8.033900
2000,7.821300
2500,7.686500
3000,7.564100
3500,7.501600
4000,7.389300
4500,7.347700
5000,7.258600




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 1h 47min 57s, sys: 37.3 s, total: 1h 48min 34s
Wall time: 1h 48min


TrainOutput(global_step=9700, training_loss=7.4240425534592465, metrics={'train_runtime': 6480.9502, 'train_samples_per_second': 95.78, 'train_steps_per_second': 1.497, 'total_flos': 2.0580929838403584e+16, 'train_loss': 7.4240425534592465, 'epoch': 1.0})

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
trainer.save_model("./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT")
# trainer.save_pretrained("./drive/MyDrive/Amharic_Sentiment_Analysis/AmBERT")

NameError: ignored

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!git init

Initialized empty Git repository in /content/.git/


In [ ]:
# trainer.push_to_hub("AmBERT")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="surafelkindu/AmBERT",
    tokenizer="surafelkindu/AmBERT"
)

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/319M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

In [ ]:
fill_mask("ጉራጌ <mask>")

[{'score': 0.007474634796380997,
  'sequence': 'ጉራጌ ላይ',
  'token': 1920,
  'token_str': 'ላይ'},
 {'score': 0.007234705146402121,
  'sequence': 'ጉራጌ ሁሉ',
  'token': 2101,
  'token_str': 'ሁሉ'},
 {'score': 0.006549467332661152,
  'sequence': 'ጉራጌ ነው',
  'token': 1916,
  'token_str': 'ነው'},
 {'score': 0.005704805254936218,
  'sequence': 'ጉራጌ ፣',
  'token': 1801,
  'token_str': '፣'},
 {'score': 0.005628969520330429,
  'sequence': 'ጉራጌ ታሪክ',
  'token': 2309,
  'token_str': 'ታሪክ'}]

In [ ]:
fill_mask("ከበደ ወደ አዲስ <mask>")

[{'score': 0.37291184067726135,
  'sequence': 'ከበደ ወደ አዲስ አበባ',
  'token': 2276,
  'token_str': 'አበባ'},
 {'score': 0.06279212981462479,
  'sequence': 'ከበደ ወደ አዲስ ወደ',
  'token': 1965,
  'token_str': 'ወደ'},
 {'score': 0.019285565242171288,
  'sequence': 'ከበደ ወደ አዲስ አዲስ',
  'token': 2174,
  'token_str': 'አዲስ'},
 {'score': 0.012872619554400444,
  'sequence': 'ከበደ ወደ አዲስ ውስጥ',
  'token': 1947,
  'token_str': 'ውስጥ'},
 {'score': 0.011713267304003239,
  'sequence': 'ከበደ ወደ አዲስ ከተማ',
  'token': 2098,
  'token_str': 'ከተማ'}]